In [5]:
import pandas as pd

# Adjust the delimiter based on your file's structure
bgame_df = pd.read_csv('test_board_game_price.csv', delimiter=',')

# Display the first few rows
print(bgame_df.head())

                                Name  Min Players  Max Players  Play Time  \
0                         Gloomhaven            1            4        120   
1          Pandemic Legacy: Season 1            2            4         60   
2                  Brass: Birmingham            2            4        120   
3                  Terraforming Mars            1            5        120   
4  Twilight Imperium: Fourth Edition            3            6        480   

   Min Age  Users Rated  Rating Average  Complexity Average  Owned Users  \
0       14        42055            8.79                3.86      68323.0   
1       13        41643            8.61                2.84      65294.0   
2       14        19217            8.66                3.91      28785.0   
3       12        64864            8.43                3.24      87099.0   
4       14        13468            8.70                4.22      16831.0   

          Domain1  above_average Bayesian Average Most Notable Genre  \
0  Strat

In [6]:
len(bgame_df)

20343

In [7]:
bgame_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20343 entries, 0 to 20342
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   20343 non-null  object 
 1   Min Players            20343 non-null  int64  
 2   Max Players            20343 non-null  int64  
 3   Play Time              20343 non-null  int64  
 4   Min Age                20343 non-null  int64  
 5   Users Rated            20343 non-null  int64  
 6   Rating Average         20343 non-null  float64
 7   Complexity Average     20343 non-null  float64
 8   Owned Users            20320 non-null  float64
 9   Domain1                10184 non-null  object 
 10  above_average          20343 non-null  bool   
 11  Bayesian Average       20127 non-null  object 
 12  Most Notable Genre     20127 non-null  object 
 13  Most Notable Mechanic  20127 non-null  object 
 14  Price                  20127 non-null  object 
dtypes:

In [9]:
bgame_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20343 entries, 0 to 20342
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   20343 non-null  object 
 1   Min Players            20343 non-null  int64  
 2   Max Players            20343 non-null  int64  
 3   Play Time              20343 non-null  int64  
 4   Min Age                20343 non-null  int64  
 5   Users Rated            20343 non-null  int64  
 6   Rating Average         20343 non-null  float64
 7   Complexity Average     20343 non-null  float64
 8   Owned Users            20320 non-null  float64
 9   Domain1                10184 non-null  object 
 10  above_average          20343 non-null  bool   
 11  Bayesian Average       20127 non-null  object 
 12  Most Notable Genre     20127 non-null  object 
 13  Most Notable Mechanic  20127 non-null  object 
 14  Price                  20127 non-null  object 
dtypes:

In [13]:
unique_domain = bgame_df['Domain1'].unique()
print(unique_domain)

['Strategy Games' 'Thematic Games' 'Family Games' 'Customizable Games'
 'Abstract Games' 'Party Games' 'Wargames' "Children's Games" nan]


In [14]:
unique_genre = bgame_df['Most Notable Genre'].unique()
print(unique_genre)

['Adventure' 'Environmental' 'Age of Reason' 'Economic' 'Civilization'
 'Card Game' 'Civil War' 'Modern Warfare' 'American West' 'Fantasy' 'Dice'
 'Ancient' 'Animals' 'Medieval' 'City Building' 'Abstract Strategy'
 'Bluffing' 'Action / Dexterity' 'Arabian' 'Industry / Manufacturing'
 'Renaissance' 'Deduction' 'Medical' 'Science Fiction' 'Aviation / Flight'
 'Exploration' 'Trains' 'Post-Napoleonic' 'Miniatures' 'Wargame'
 'Party Game' 'Movies / TV / Radio theme' 'Horror' 'Real-time' 'Racing'
 nan 'Humor' 'Maze' 'Spies/Secret Agents' 'Collectible Components'
 'Nautical' 'Fighting' 'Prehistoric' 'Educational' 'Math' 'Travel'
 'Mythology' 'Negotiation' 'Political' "Children's Game" 'Napoleonic'
 'Farming' 'Transportation' 'Territory Building' 'American Civil War'
 'Parsing Error' 'American Indian Wars' 'Comic Book / Strip' 'Mafia'
 'Electronic' 'Not available' 'Puzzle' 'Murder/Mystery' 'Novel-based'
 'Sports' 'Korean War' 'Print & Play' 'Book' 'American Revolutionary War'
 'Religious' 'Vid

In [15]:
unique_mechanic = bgame_df['Most Notable Mechanic'].unique()
print(unique_mechanic)

['Action Queue' 'Action Points' 'Hand Management' 'Closed Drafting'
 'Action Drafting' 'End Game Bonuses' 'Area Majority / Influence'
 'Action/Event' 'Deck, Bag, and Pool Building' 'Action Retrieval'
 'Dice Rolling' 'Automatic Resource Growth' 'Contracts'
 'Campaign / Battle Card Driven' 'Card Play Conflict Resolution'
 'Delayed Purchase' 'Area Movement' 'Cooperative Game'
 'Auction: Turn Order Until Pass' 'Bias' 'Communication Limits'
 'Commodity Speculation' 'Follow' 'Catch the Leader' 'Flicking'
 'Auction/Bidding' 'Chit-Pull System' 'Auction: Fixed Placement'
 'Enclosure' 'Grid Coverage' 'Bingo' 'Connections' 'Finale Ending'
 'Advantage Token' 'Alliances' 'Increase Value of Unchosen Resources'
 'Auction: Dutch' 'Events' 'Betting and Bluffing' 'Grid Movement'
 'Auction: Once Around' 'Hidden Roles' nan 'Acting' 'Investment'
 'Open Drafting' 'Pattern Recognition' 'Hexagon Grid' 'Order Counters'
 'Critical Hits and Failures' 'Race' 'Deduction' 'Chaining'
 'Point to Point Movement' 'Pick

In [30]:
domain_stats = bgame_df.groupby('Domain1').agg({
    'Rating Average': 'mean', 
    'Owned Users': 'sum'
}).sort_values(by='Rating Average', ascending=False)

# Display the result
print(domain_stats)

                    Rating Average  Owned Users
Domain1                                        
Strategy Games            7.028981    7742493.0
Wargames                  6.852060    1723997.0
Thematic Games            6.839453    3174966.0
Family Games              6.424877    8060172.0
Customizable Games        6.370102     664351.0
Party Games               6.309691    1925037.0
Abstract Games            6.230140    1445470.0
Children's Games          5.516485     596811.0


In [28]:
genre_stats = bgame_df.groupby('Most Notable Genre').agg({
    'Rating Average': 'mean', 
    'Owned Users': 'sum'
}).sort_values(by='Rating Average', ascending=False)

# Display the result
print(genre_stats)

                            Rating Average  Owned Users
Most Notable Genre                                     
Expansion for Base-game           7.461111       3111.0
Book                              7.417609      31702.0
American Revolutionary War        7.157500       5549.0
Miniatures                        7.151044     151635.0
Napoleonic                        7.066940      99962.0
...                                    ...          ...
Movies / TV / Radio theme         5.715728     137968.0
Zombies                           5.652000       4462.0
Children's Game                   5.577347     390353.0
Mature / Adult                    5.451000       9235.0
Trivia                            5.413548      15977.0

[83 rows x 2 columns]


In [29]:
mech_stats = bgame_df.groupby('Most Notable Mechanic').agg({
    'Rating Average': 'mean', 
    'Owned Users': 'sum'
}).sort_values(by='Rating Average', ascending=False)

# Display the result
print(mech_stats)

                           Rating Average  Owned Users
Most Notable Mechanic                                 
Die Icon Resolution              7.723333        859.0
Automatic Resource Growth        7.647778     185282.0
Finale Ending                    7.580000      38202.0
Auction: Fixed Placement         7.540000      27843.0
Measurement Movement             7.496667       1072.0
...                                   ...          ...
Single Loser Game                5.323333       3042.0
Passed Action Token              5.290000        275.0
Roll / Spin and Move             5.226841     207994.0
Questions and Answers            5.100000       1509.0
Auction: English                 4.680000      41300.0

[160 rows x 2 columns]
